<a href="https://colab.research.google.com/github/venkatrajan247/Citizen-Science-EnviroTrack/blob/main/Citizen_Science_EnviroTrack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install "torch==2.4.0" "xformers==0.0.27.post2" triton torchvision torchaudio
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"


In [ ]:
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-qbfg43d1/unsloth_a2c2145695044186ae1864094d6bccb0
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-qbfg43d1/unsloth_a2c2145695044186ae1864094d6bccb0
  Resolved https://github.com/unslothai/unsloth.git to commit 2ff5dc1a8de1614994a275785b7b64fb4db8cb5d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
import torch
from unsloth import PatchDPOTrainer
from transformers import TrainingArguments
from trl import DPOTrainer
from unsloth import FastLanguageModel
from datasets import load_dataset
from unsloth import FastLanguageModel  # ✅ Import FastLanguageModel

PatchDPOTrainer()

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

In [ ]:
# Load the dataset from the specified Hugging Face repository and split it into the 'train' set
dataset = load_dataset("Venkatrajan247/citizen-science-monitoring", split='train')

# Rename the 'text' column to 'chosen' for clarity and to align with the expected column naming
dataset = dataset.rename_column('text', 'chosen')

# Rename the 'rejected_text' column to 'rejected' for consistency with the 'chosen' column
dataset = dataset.rename_column('rejected_text', 'rejected')

README.md:   0%|          | 0.00/4.48k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# print(dataset)
print(dataset)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]


DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 900
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 100
    })
})


In [ ]:
# Define the maximum sequence length for the model to process
max_seq_length = 4096

# Specify the data type for the model (set to None for default behavior)
dtype = None

# Enable 4-bit quantization to reduce memory usage; set to False if higher precision is required
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    attn_implementation="flash_attention_2"  # Faster training & inference
)


==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [ ]:
# Apply PEFT (Parameter-Efficient Fine-Tuning) to the quantized model using LoRA
model = FastLanguageModel.get_peft_model(
    model,  # The pre-trained and potentially quantized model

    # The rank of the low-rank decomposition matrices for LoRA. Higher values allow
    # the model to capture more task-specific information (common values: 8, 16, 32, 64, 128).
    r=64,

    # Specify the target modules where LoRA should be applied, such as key, value,
    # query projections, and other attention-related layers.
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention projection layers
        "gate_proj", "up_proj", "down_proj",    # Feed-forward network layers
    ],

    # LoRA-specific hyperparameters:
    lora_alpha=64,  # Scaling factor for the LoRA update
    lora_dropout=0,  # Dropout for LoRA layers; set to 0 for no dropout
    bias="none",  # Bias handling strategy; options are "none", "all", or "lora_only"

    # Enable gradient checkpointing for memory efficiency during training.
    # "unsloth" is a custom value, likely to support longer context lengths.
    use_gradient_checkpointing="unsloth",

    random_state=3407,  # Random seed for reproducibility during training

    # Optional configurations for advanced techniques:
    use_rslora=False,  # Flag for whether to use Randomized Singular LoRA (RsLoRA)
    loftq_config=None,  # Configuration for LoFT-Q (LoRA for Fine-Tuning Quantized models)
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
!pip show unsloth


Name: unsloth
Version: 2025.3.19
Summary: 2-5X faster LLM finetuning
Home-page: http://www.unsloth.ai
Author: Unsloth AI team
Author-email: info@unsloth.ai
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: 
Required-by: 


In [ ]:
# Initialize the DPOTrainer (Direct Preference Optimization Trainer) to fine-tune the model
dpo_trainer = DPOTrainer(
    model=model,  # The model to be fine-tuned (already loaded and potentially quantized with LoRA)

    # Define training arguments for fine-tuning
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of samples processed per device in one forward/backward pass
        gradient_accumulation_steps=3,  # Accumulate gradients over multiple steps to simulate a larger batch size
        warmup_ratio=0.1,  # Fraction of total steps used for learning rate warmup
        num_train_epochs=1,  # Number of training epochs
        learning_rate=5e-6,  # Initial learning rate for the optimizer
        fp16=not torch.cuda.is_bf16_supported(),  # Use FP16 if BF16 is not supported
        bf16=torch.cuda.is_bf16_supported(),  # Use BF16 precision if supported by the GPU
        logging_steps=10,  # Log training progress every 10 steps
        optim="adamw_8bit",  # Use AdamW optimizer with 8-bit precision for memory efficiency
        weight_decay=0.001,  # Weight decay for regularization
        lr_scheduler_type="linear",  # Use a linear learning rate scheduler
        seed=42,  # Set random seed for reproducibility
        report_to="none",  # Disable reporting to external tools (e.g., WandB or TensorBoard)
        output_dir="outputs",  # Directory to save the model and training artifacts
    ),

    beta=0.1,  # Beta parameter for DPO; balances the loss for optimization
    train_dataset=dataset["train"],  # Training dataset
    eval_dataset=dataset["test"],  # Evaluation dataset
    tokenizer=tokenizer,  # Tokenizer corresponding to the model
    max_length=4096,  # Maximum token length for input sequences
    max_prompt_length=512,  # Maximum token length for prompts within input sequences
)

Extracting prompt in train dataset (num_proc=2):   0%|          | 0/900 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/900 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/900 [00:00<?, ? examples/s]

Extracting prompt in eval dataset (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Applying chat template to eval dataset (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
dpo_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 900 | Num Epochs = 1 | Total steps = 150
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 3
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 3 x 1) = 6
 "-____-"     Trainable parameters = 167,772,160/7,000,000,000 (2.40% trained)


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / chosen,logps / rejected,logits / chosen,logits / rejected,eval_logits / chosen,eval_logits / rejected,nll_loss,aux_loss
10,0.550700,0.126060,-0.239979,0.800000,0.366039,-94.275459,-95.433029,-3.457104,-3.281299,0,0,0,0
20,0.044500,1.411379,-2.916584,1.000000,4.327962,-86.015160,-126.111473,-3.544629,-3.357262,No Log,No Log,No Log,No Log
30,0.000300,2.666580,-6.037170,1.000000,8.703750,-74.881096,-158.169998,-3.592288,-3.386767,No Log,No Log,No Log,No Log
40,0.000000,3.121093,-7.480208,1.000000,10.601302,-67.385704,-171.693405,-3.587993,-3.403439,No Log,No Log,No Log,No Log
50,0.000000,3.335104,-7.855374,1.000000,11.190478,-67.737999,-176.468460,-3.587952,-3.396444,No Log,No Log,No Log,No Log
60,0.000000,3.464488,-7.913680,1.000000,11.378168,-67.719536,-175.994812,-3.583613,-3.406558,No Log,No Log,No Log,No Log
70,0.000000,3.332936,-7.946290,1.000000,11.279225,-62.489117,-173.932526,-3.593021,-3.356974,No Log,No Log,No Log,No Log
80,0.000000,3.336592,-8.020766,1.000000,11.357357,-60.578430,-171.622818,-3.586407,-3.390167,No Log,No Log,No Log,No Log
90,0.000000,3.283423,-8.168480,1.000000,11.451903,-64.920319,-176.187042,-3.603371,-3.391474,No Log,No Log,No Log,No Log
100,0.000000,3.348545,-8.284236,1.000000,11.632782,-65.344116,-177.468994,-3.591013,-3.422018,No Log,No Log,No Log,No Log


TrainOutput(global_step=150, training_loss=0.03971250703863916, metrics={'train_runtime': 760.647, 'train_samples_per_second': 1.183, 'train_steps_per_second': 0.197, 'total_flos': 0.0, 'train_loss': 0.03971250703863916, 'epoch': 1.0})

In [ ]:
from huggingface_hub import create_repo

repo_name = "citizen-science-monitoring-dpo-trained"
username = "Venkatrajan247"  # Replace with your HF username

create_repo(f"{username}/{repo_name}", repo_type="model", exist_ok=True)


RepoUrl('https://huggingface.co/Venkatrajan247/citizen-science-monitoring-dpo-trained', endpoint='https://huggingface.co', repo_type='model', repo_id='Venkatrajan247/citizen-science-monitoring-dpo-trained')

In [ ]:
model.save_pretrained("citizen-science-monitoring-dpo-trained")
tokenizer.save_pretrained("citizen-science-monitoring-dpo-trained")

('citizen-science-monitoring-dpo-trained/tokenizer_config.json',
 'citizen-science-monitoring-dpo-trained/special_tokens_map.json',
 'citizen-science-monitoring-dpo-trained/tokenizer.model',
 'citizen-science-monitoring-dpo-trained/added_tokens.json',
 'citizen-science-monitoring-dpo-trained/tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
from transformers import AutoTokenizer

# Define your Hugging Face repo name
repo_name = "Venkatrajan247/citizen-science-monitoring-dpo-trained"  # Update with your username

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("./citizen-science-monitoring-dpo-trained")

# Push to Hugging Face Hub
tokenizer.push_to_hub(repo_name)


README.md:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Venkatrajan247/citizen-science-monitoring-dpo-trained/commit/9fa562d935859f5ad07e884bb57cfa786d3fd521', commit_message='Upload tokenizer', commit_description='', oid='9fa562d935859f5ad07e884bb57cfa786d3fd521', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Venkatrajan247/citizen-science-monitoring-dpo-trained', endpoint='https://huggingface.co', repo_type='model', repo_id='Venkatrajan247/citizen-science-monitoring-dpo-trained'), pr_revision=None, pr_num=None)

In [ ]:
model.push_to_hub(repo_name)


  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Venkatrajan247/citizen-science-monitoring-dpo-trained


In [ ]:
from unsloth import FastLanguageModel
import transformers

In [ ]:
# Load the fine-tuned model and tokenizer from the specified directory
# The model is loaded with 4-bit quantization enabled for memory efficiency
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="citizen-science-monitoring-dpo-trained",  # Path to the saved fine-tuned model
    max_seq_length=4096,  # Set the maximum sequence length for processing inputs
    load_in_4bit=True,  # Enable 4-bit quantization to reduce memory usage
)

# Prepare the model for inference (set it to evaluation mode)
FastLanguageModel.for_inference(model)  # This configures the model for inference without further training

==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.4.0+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.0.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load citizen-science-monitoring-dpo-trained as a legacy tokenizer.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32768, 4096, padding_idx=770)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [ ]:
def predict(input_prompt):
    """
    Generate a response based on the input prompt using a pre-trained language model.

    Parameters:
        input_prompt (str): The user's query or instruction.

    Returns:
        str: The model-generated response.
    """

    # Ensure model and tokenizer are on the correct device (GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Define conversation context
    messages = [
        {"role": "system", "content": "You are a helpful assistant that provides security recommendations for organizations."},
        {"role": "user", "content": input_prompt}
    ]

    # Convert messages into the model's chat format
    formatted_input = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    # Tokenize input and move it to the correct device
    model_inputs = tokenizer([formatted_input], return_tensors="pt").to(device)

    # Generate response with controlled parameters
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512,  # Reduced from 4096 for efficiency
        temperature=0.2  # Controls randomness; lower = more deterministic
    )

    # Remove input tokens from output (to get only generated response)
    response_ids = [
        output[len(input_seq):] for input_seq, output in zip(model_inputs.input_ids, generated_ids)
    ]

    # Decode response into human-readable text
    response = tokenizer.batch_decode(response_ids, skip_special_tokens=True)[0]

    return response

In [ ]:
from IPython.display import display, Markdown

def format_response(text):
    display(Markdown(f"**Response:**\n\n{text}"))

input_prompt = "How should invasive species be mapped?"
response = predict(input_prompt)
format_response(response)


**Response:**

To map invasive species effectively in an organizational context, a comprehensive approach is recommended. Here's a step-by-step guide on how to map invasive species:

1. Identify the target species: Research and identify the invasive species that are of concern for your organization. Gather information about their characteristics, life cycle, and distribution.

2. Establish a baseline: Collect data on the current distribution of the invasive species within your organization's area of interest. This can be done through field surveys, remote sensing, or by analyzing existing data sources such as government databases, academic research, and citizen science projects.

3. Create a mapping tool: Develop a GIS (Geographic Information System) or a similar mapping tool to visualize the distribution of the invasive species. This tool should allow you to input data, analyze patterns, and create maps that can be easily understood by stakeholders.

4. Collect data systematically: Establish a standardized data collection protocol to ensure consistency in data gathering. This protocol should include details such as the location, date, and method of observation, as well as the identification of the invasive species.

5. Conduct regular surveys: Schedule regular surveys to monitor the spread of the invasive species. Surveys can be conducted at fixed intervals or in response to changes in the environment or reported sightings.

6. Analyze data: Analyze the collected data to identify trends, hotspots, and potential areas of concern. This analysis can help prioritize management efforts and inform decision-making.

7. Collaborate with stakeholders: Engage with relevant stakeholders, such as local communities, government agencies, and other organizations, to share data and collaborate on management strategies.

8. Update the map: Regularly update the map as new data is collected and analyzed. This will help keep the map current and ensure that management efforts are focused on the most up-to-date information.

9. Develop management strategies: Based on the data and analysis, develop strategies to manage the invasive species. This may include control measures, prevention efforts, and education and outreach programs.

10. Monitor and evaluate: Continuously monitor the effectiveness of management strategies and adjust them as necessary. Evaluate the impact of management efforts on the distribution and abundance of the invasive species.

By following these steps, your organization can effectively

In [ ]:
from IPython.display import display, Markdown

def format_response(text):
    display(Markdown(f"**Response:**\n\n{text}"))

input_prompt = "'Can you summarize your response to \"How should invasive species be mapped?\"in one sentence?'"
response = predict(input_prompt)
format_response(response)

**Response:**

To map invasive species effectively, utilize a combination of GIS technology, collaborative databases, and regular field surveys to track their distribution, impact, and potential spread within an organization's ecosystem.

In [ ]:
from IPython.display import display, Markdown

def format_response(text):
    display(Markdown(f"**Response:**\n\n{text}"))

input_prompt = "what are the seven wonders in world and explain their interesting thing"
response = predict(input_prompt)
format_response(response)

**Response:**

The Seven Wonders of the Ancient World are a list of remarkable constructions of the classical antiquity, selected by the Greek philosopher Phidias, who was a student of Socrates. Here's a brief description of each:

1. Great Pyramid of Giza, Egypt: The Great Pyramid is the oldest and largest of the three pyramids in the Giza pyramid complex. It was built as a tomb for Pharaoh Khufu (Cheops) around 2580-2560 BC. The pyramid's base covers 13 acres and it originally was 481 feet tall.

2. Hanging Gardens of Babylon, Babylon (present-day Iraq): These were supposedly a series of terraced gardens filled with lush vegetation, built by King Nebuchadnezzar II around 600 BC to please his wife, who missed her homeland's greenery. Unfortunately, no archaeological evidence has been found to confirm their existence.

3. Temple of Artemis at Ephesus, Turkey: Dedicated to the goddess Artemis, this temple was one of the seven wonders of the ancient world. It was built around 550 BC and was considered the largest temple in the Greek world. It was destroyed and rebuilt several times, with the last version being destroyed by a fire in 401 AD.

4. Statue of Zeus at Olympia, Greece: This colossal bronze statue of Zeus, the king of the Greek gods, was sculpted by Phidias around 435 BC. It stood approximately 40 feet tall and was housed in the Temple of Zeus at Olympia. The statue is now lost, but descriptions and surviving copies give us an idea of its grandeur.

5. Mausoleum at Halicarnassus, Turkey: This tomb was built between 351 and 350 BC for Mausolus, a Persian satrap, and his wife Artemisia. The structure was an elaborate tomb with a Doric portico, Ionic frieze, and a pyramid-like roof. The term "mausoleum" is derived from this structure.

6. Colossus of Rhodes, Greece: This was a coloss

In [ ]:
!pip install streamlit transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.7 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import upload_file

upload_file(
    path_or_fileobj="citizen-science-monitoring-dpo-trained/tokenizer.model",
    path_in_repo="tokenizer.model",
    repo_id="Venkatrajan247/citizen-science-monitoring-dpo-trained",
    repo_type="model"
)


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Venkatrajan247/citizen-science-monitoring-dpo-trained/commit/9ecb1ca7667ed8c91898701b2243fb9d7db72979', commit_message='Upload tokenizer.model with huggingface_hub', commit_description='', oid='9ecb1ca7667ed8c91898701b2243fb9d7db72979', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Venkatrajan247/citizen-science-monitoring-dpo-trained', endpoint='https://huggingface.co', repo_type='model', repo_id='Venkatrajan247/citizen-science-monitoring-dpo-trained'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer

model_name = "Venkatrajan247/citizen-science-monitoring-dpo-trained"

tokenizer = AutoTokenizer.from_pretrained(model_name, force_download=True, use_fast=False)


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.67M [00:00<?, ?B/s]